# __Daily Social Network Analysis__

In [295]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import re

In [116]:
day = pd.read_csv('data/tweets/week_10/day_2_2021-06-28.csv', index_col=0)
day.replace({np.nan: False}, inplace=True)
day.shape

(90396, 27)

In [117]:
day.columns

Index(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang',
       'source', 'sourceUrl', 'sourceLabel', 'outlinks', 'tcooutlinks',
       'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
       'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags',
       'cashtags'],
      dtype='object')

In [122]:
day.sample()[['user', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers']]

,user,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers
47511,"{'username': 'koveirus', 'id': 287391903, 'dis...",False,False,1409507000329908224.0,"{'username': 'PastorEliasMau1', 'id': 11799253...","[{'username': 'PastorEliasMau1', 'id': 1179925..."


In [123]:
day.iloc[47511]['url']

'https://twitter.com/koveirus/status/1409514118781018112'

In [125]:
reply_df = day.query('inReplyToUser != False')
reply_df.shape

(46294, 27)

In [232]:
reply_df = reply_df[['content', 'user', 'inReplyToUser']]
reply_df.head()

,content,user,inReplyToUser
1,@CarlaZambelli38 A esquerdalhada está triste c...,"{'username': 'ArthouEloi', 'id': 1366052843233...","{'username': 'CarlaZambelli38', 'id': 97512772..."
2,@revistaoeste #BolsonaroAte2026 #GloboLixo #ag...,"{'username': 'BolsoNABOneles', 'id': 118898189...","{'username': 'revistaoeste', 'id': 12229398963..."
3,@CarlaZambelli38 @jairbolsonaro @mqueiroga2 @L...,"{'username': 'MariliaRezend11', 'id': 12563324...","{'username': 'CarlaZambelli38', 'id': 97512772..."
4,"@GabriellaPsicom @taliriapetrone Vá se tratar,...","{'username': 'MiriamNetto', 'id': 105295393698...","{'username': 'GabriellaPsicom', 'id': 97141617..."
5,@OmarAzizSenador Que vergonha eu sinto de você...,"{'username': 'MB_Marcos_', 'id': 1289048882459...","{'username': 'OmarAzizSenador', 'id': 24459217..."


In [350]:
user_regex_str = "[{'username':]+\s*[']+\w*"
id_regex_str = "[,]\s*['id':]+\s*\d*"
user_regex = re.compile(user_regex_str)
id_regex = re.compile(id_regex_str)
regex_number = re.compile('\d+')

def return_username(dict_str):
    return user_regex.search(dict_str).group().replace("{'username': '", '')

def return_id(dict_str):
    return regex_number.search(id_regex.search(dict_str).group()).group()


In [356]:
reply_df['username'] = reply_df['user'].map(lambda x: return_username(x))
reply_df['reply_to'] = reply_df['inReplyToUser'].map(lambda x: return_username(x))
reply_df['user_id'] = reply_df['user'].map(lambda x: int(return_id(x)))
reply_df['reply_to_id'] = reply_df['inReplyToUser'].map(lambda x: int(return_id(x)))

reply_df.head()

C:\Users\LUCASR~1\AppData\Local\Temp/ipykernel_544/41477684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reply_df['username'] = reply_df['user'].map(lambda x: return_username(x))
C:\Users\LUCASR~1\AppData\Local\Temp/ipykernel_544/41477684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reply_df['reply_to'] = reply_df['inReplyToUser'].map(lambda x: return_username(x))
C:\Users\LUCASR~1\AppData\Local\Temp/ipykernel_544/41477684.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,content,user,inReplyToUser,username,reply_to,user_id,reply_to_id
1,@CarlaZambelli38 A esquerdalhada está triste c...,"{'username': 'ArthouEloi', 'id': 1366052843233...","{'username': 'CarlaZambelli38', 'id': 97512772...",ArthouEloi,CarlaZambelli38,1366052843233435650,975127727501185025
2,@revistaoeste #BolsonaroAte2026 #GloboLixo #ag...,"{'username': 'BolsoNABOneles', 'id': 118898189...","{'username': 'revistaoeste', 'id': 12229398963...",BolsoNABOneles,revistaoeste,1188981896056922112,1222939896349970432
3,@CarlaZambelli38 @jairbolsonaro @mqueiroga2 @L...,"{'username': 'MariliaRezend11', 'id': 12563324...","{'username': 'CarlaZambelli38', 'id': 97512772...",MariliaRezend11,CarlaZambelli38,1256332466480087043,975127727501185025
4,"@GabriellaPsicom @taliriapetrone Vá se tratar,...","{'username': 'MiriamNetto', 'id': 105295393698...","{'username': 'GabriellaPsicom', 'id': 97141617...",MiriamNetto,GabriellaPsicom,1052953936985038849,971416178135633921
5,@OmarAzizSenador Que vergonha eu sinto de você...,"{'username': 'MB_Marcos_', 'id': 1289048882459...","{'username': 'OmarAzizSenador', 'id': 24459217...",MB_Marcos_,OmarAzizSenador,1289048882459140096,2445921702


In [368]:
nodes_id = pd.concat([reply_df['user_id'], reply_df['reply_to_id']], axis=0)
nodes_name = pd.concat([reply_df['username'], reply_df['reply_to']], axis=0)
nodes = pd.concat([nodes_id, nodes_name], axis=1)
nodes.drop_duplicates(inplace=True)
nodes.shape

(14672, 2)

In [389]:
edges = reply_df[['user_id', 'reply_to_id']]
edges.shape

(46294, 2)

In [399]:
G = nx.DiGraph()

for node in nodes.iterrows():
    G.add_node(node[1][0], username=node[1][1])
    
for edge in edges.iterrows():
    G.add_edge(edge[1]['user_id'], edge[1]['reply_to_id'])

In [405]:
# metrics
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweennes_centrality = nx.betweenness_centrality(G)

In [406]:
indegree = nx.in_degree_centrality(G)

In [440]:
clustering = nx.clustering(G)

In [443]:
sorted_indegree = sorted(indegree.items(), key=lambda x: x[1], reverse=True)
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)
sorted_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)
sorted_betweennes = sorted(betweennes_centrality.items(), key=lambda x: x[1], reverse=True)
sorted_clustering = sorted(clustering.items(), key=lambda x: x[1], reverse=True)

In [445]:
print(f'Highest Indegree - id: {sorted_indegree[0][0]} value: {sorted_indegree[0][1]}'
     f'\nHighest Degree - id: {sorted_degree[0][0]} value: {sorted_degree[0][1]}'
     f'\nHighest Closeness - id: {sorted_closeness[0][0]} value: {sorted_closeness[0][1]}'
     f'\nHighest Betweennes - id: {sorted_betweennes[0][0]} value: {sorted_betweennes[0][1]}'
     f'\nHighest Clustering coefficient - id: {sorted_clustering[0][0]} value: {sorted_clustering[0][1]}')

Highest Indegree - id: 75264300 value: 0.22344197463521068
Highest Degree - id: 75264300 value: 0.22351015955270695
Highest Closeness - id: 75264300 value: 0.2309729183049628
Highest Betweennes - id: 391606213 value: 0.008000322407180304
Highest Clustering coefficient - id: 376414840 value: 1.0


In [446]:
reply_df.query('user_id == 376414840')

,content,user,inReplyToUser,username,reply_to,user_id,reply_to_id
437,@ClaraFernanndez #RenanSabiaDeTudo sim claro q...,"{'username': 'rcbarbos', 'id': 376414840, 'dis...","{'username': 'ClaraFernanndez', 'id': 71626385...",rcbarbos,ClaraFernanndez,376414840,71626385
11871,@CrisCapablanca @GaleriaTurandot #RenanSabiaDe...,"{'username': 'rcbarbos', 'id': 376414840, 'dis...","{'username': 'CrisCastilhano', 'id': 125051874...",rcbarbos,CrisCastilhano,376414840,1250518742435606528


In [410]:
# visualization
#nx.draw_spring(G)
#plt.show()

In [457]:
nx.density(G)

0.0001741604666368042